<a href="https://colab.research.google.com/github/yakaralar/IE492/blob/master/meta_brute_tree-kubi-the-last.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:

import os
import glob
import shutil
os.listdir()

['.config', 'drive', 'sample_data']

In [3]:
os.listdir("drive/My Drive/colab_datas/IE 492")

['Comparison of methods.pdf',
 '1-s2.0-S1364032114000914-main.pdf',
 '1-s2.0-S0360544209002539-main.pdf',
 '1-s2.0-S1876610211043116-main.pdf',
 '1-s2.0-S0305048311001423-main.pdf',
 'weather_turkey_processed.zip',
 'Meta_L_makaleler',
 'all_data.feather',
 'UECM-15012015-20022020.csv',
 'RealTimeConsumption-01122014-09032020.csv',
 'updated_support-date_based_information.r',
 'ozel_gunler',
 'RealTimeConsumption_unholidayized.csv',
 'RealTimeConsumption_unhol_decomposed.csv',
 'RealTimeConsumption_holiday.csv',
 'kubi_preds',
 'arima_preds.csv',
 'arima_preds_datetime.csv',
 'gpw_v4_admin_unit_center_points_population_estimates_rev11_tur.csv',
 'gpw-v4-country-level-summary-rev11.xlsx',
 'gpw-v4-documentation-rev11.pdf',
 'kubi_weather_data_20151231-2020-04-02.csv',
 'kubi_temp_humid_wind_data_20151231-2020-04-02.csv',
 'lasso_coeffs_new.csv']

In [4]:
shutil.copy("drive/My Drive/colab_datas/IE 492/RealTimeConsumption_unholidayized.csv","./")
shutil.copy("drive/My Drive/colab_datas/IE 492/RealTimeConsumption-01122014-09032020.csv","./")
shutil.copy("drive/My Drive/colab_datas/IE 492/RealTimeConsumption_holiday.csv","./")
shutil.copy("drive/My Drive/colab_datas/IE 492/kubi_temp_humid_wind_data_20151231-2020-04-02.csv","./")

'./kubi_temp_humid_wind_data_20151231-2020-04-02.csv'

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
%matplotlib inline
from tqdm.notebook import tqdm

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# eval func

In [6]:
!pip install graphviz
!pip install sklearn --upgrade
from sklearn.metrics import mean_squared_error,mean_squared_log_error,mean_absolute_error
from sklearn.tree import DecisionTreeRegressor,plot_tree

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

eval_df=[]
def eval_function(name_,y_true, y_pred,valid_df):
    """
    name_:kullanılan modelin ismi
    y_true:actual values
    y_pred:predicted values
    valid_df:modele verdiginiz sekliyle validation dataframe'i ve yanına datetime'ı
    """
    ap="drive/My Drive/colab_datas/IE 492/kubi_preds/eval_plots/"+name_
    if not os.path.exists(ap):
        os.mkdir(ap)
    rmse_=np.sqrt(mean_squared_error(y_true,y_pred))
    rmsle_=np.sqrt(mean_squared_log_error(y_true,y_pred))
    mae_=mean_absolute_error(y_true,y_pred)
    mape_=mean_absolute_percentage_error(y_true, y_pred)
    mapes_each_period=[mean_absolute_percentage_error(y_true[i],y_pred[i]) for i in range(len(y_true))]
    mapes_each_period=pd.DataFrame(mapes_each_period,columns=["mapes"])
    mapes_each_period["Datetime"]=valid_df["Datetime"].iloc[-len(mapes_each_period):]
    print("RMSE: ",rmse_)
    print("RMSLE: ",rmsle_)
    print("MAE: ",mae_)
    print("MAPE: ",mape_)
    fig=plt.figure(figsize=(16,10))
    sns.scatterplot(x=y_pred,y=y_true)
    fig.savefig("drive/My Drive/colab_datas/IE 492/kubi_preds/eval_plots/"+name_+"/actual_vs_predicted.png")
    plt.pause(0.01)
    fig=plt.figure(figsize=(16,10))
    sns.residplot(y_true,y_pred)
    plt.pause(0.01)
    fig.savefig("drive/My Drive/colab_datas/IE 492/kubi_preds/eval_plots/"+name_+"/residplot.png")
    nchunks=6 # plot count
    fig,ax=plt.subplots(nrows=32,figsize=(32,125))
    valid_copy=valid_df.reset_index(drop=True).copy()

    for i,datetime_ in enumerate(np.array_split(valid_copy["Datetime"],32)):
        sns.lineplot(y=y_true[datetime_.index.tolist()],x=datetime_,color="green",ax=ax[i])
        sns.lineplot(y=y_pred[datetime_.index.tolist()],x=datetime_,color="red",ax=ax[i])
    fig.savefig("drive/My Drive/colab_datas/IE 492/kubi_preds/eval_plots/"+name_+"/lineplot_pred_act_{}.png".format(i))

    fig,ax=plt.subplots(nrows=nchunks,figsize=(32,25))
    for i,datetime_ in enumerate(np.array_split(mapes_each_period,nchunks)):
        sns.lineplot(y="mapes",x="Datetime",data=datetime_,ax=ax[i])
    fig.savefig("drive/My Drive/colab_datas/IE 492/kubi_preds/eval_plots/"+name_+"/lineplot_errors.png")

    # plt.pause(0.01)
    # res_model=DecisionTreeRegressor(max_depth=4)
    # _, ax = plt.subplots(figsize=(60, 60))  # whatever size you want
    # fig=res_model.fit(valid_df.drop(columns=["Datetime"]),y_true-y_pred)
    # plot_tree(fig,ax=ax,feature_names=valid_df.drop(columns=["Datetime"]).columns,fontsize=16)
    # plt.pause(0.01)
    # ## saatlik mape tanımla
    return name_, rmse_,rmsle_,mae_,mape_

Requirement already up-to-date: sklearn in /usr/local/lib/python3.6/dist-packages (0.0)


# Consumption df

In [0]:
# # bruuuuuuuh
# cons_holiday_binary=pd.read_csv("RealTimeConsumption_holiday.csv",encoding='latin-1')
# cons_holiday_binary["Datetime"]=cons_holiday_binary["Date"]+" "+cons_holiday_binary["Hour"]
# cons_holiday_binary["Datetime"]=pd.to_datetime(cons_holiday_binary["Datetime"],format="%d.%m.%Y %H:%M")
# cons_holiday_binary.head()

In [0]:
# catboostu 1 aylık yap

In [9]:
preds_paths=glob.glob("drive/My Drive/colab_datas/IE 492/kubi_preds/*.csv")
preds_paths

['drive/My Drive/colab_datas/IE 492/kubi_preds/LR_2.csv',
 'drive/My Drive/colab_datas/IE 492/kubi_preds/naive_2.csv',
 'drive/My Drive/colab_datas/IE 492/kubi_preds/LR_2_7_w_dates.csv',
 'drive/My Drive/colab_datas/IE 492/kubi_preds/naive_7.csv',
 'drive/My Drive/colab_datas/IE 492/kubi_preds/LR_2_7.csv',
 'drive/My Drive/colab_datas/IE 492/kubi_preds/LR_7.csv',
 'drive/My Drive/colab_datas/IE 492/kubi_preds/ygur_rima_preds_datetime.csv',
 'drive/My Drive/colab_datas/IE 492/kubi_preds/Cat1_48hours.csv',
 'drive/My Drive/colab_datas/IE 492/kubi_preds/Cat2_categorical_48hours.csv',
 'drive/My Drive/colab_datas/IE 492/kubi_preds/Cat1_24hours.csv',
 'drive/My Drive/colab_datas/IE 492/kubi_preds/Cat2_categorical_24hours.csv',
 'drive/My Drive/colab_datas/IE 492/kubi_preds/Keras_Basic_dense_24.csv',
 'drive/My Drive/colab_datas/IE 492/kubi_preds/Keras_Basic_dense_48.csv',
 'drive/My Drive/colab_datas/IE 492/kubi_preds/HW_preds_48hours.csv',
 'drive/My Drive/colab_datas/IE 492/kubi_preds/HW_

In [0]:
#a=pd.read_csv("drive/My Drive/colab_datas/IE 492/kubi_preds/ygur_rima_preds_datetime.csv")
#a["Datetime"]=pd.to_datetime(a["Datetime"],format="%d/%m/%Y %H:%M")
#a=a.iloc[:,:-1]
#a.columns=["Datetime","Arima"]
#a.to_csv("drive/My Drive/colab_datas/IE 492/kubi_preds/ygur_rima_preds_datetime.csv",index=False)

In [0]:
#path_="drive/My Drive/colab_datas/IE 492/kubi_preds/erdem_HW_preds_new.csv"
#a=pd.read_csv(path_)
#a["Datetime"]=pd.to_datetime(a["Datetime"])
#a.loc[a["Datetime"]>=pd.to_datetime("2019-02-01 00:00:00")][["Datetime","HW_unhol_reg"]].to_csv(path_,index=False)

In [12]:
pred_dfs=[]
for i,path_ in enumerate(preds_paths):
    df=pd.read_csv(path_)
    if "HW" in path_:
        df=df.loc[df["Datetime"].between('2019-02-01','2020-03-05')].reset_index(drop=True)
    elif "rima" in path_:
        df["Datetime"]=pd.date_range('2019-02-01','2020-03-04',freq="H")[:-1]
    df.columns=["Datetime_"+str(i),path_.split("/")[-1].strip(".csv")]
    pred_dfs.append(df)
preds=pd.concat(pred_dfs,axis=1)
preds

,Datetime_0,LR_2,Datetime_1,naive_2,Datetime_2,LR_2_7_w_date,Datetime_3,naive_7,Datetime_4,LR_2_7,Datetime_5,LR_7,Datetime_6,ygur_rima_preds_datetime,Datetime_7,Cat1_48hour,Datetime_8,Cat2_categorical_48hour,Datetime_9,Cat1_24hour,Datetime_10,Cat2_categorical_24hour,Datetime_11,Keras_Basic_dense_24,Datetime_12,Keras_Basic_dense_48,Datetime_13,HW_preds_48hour,Datetime_14,HW_preds_24hour
0,2019-02-01 00:00:00,31744.801869,2019-02-01 00:00:00,31427.68,2019-02-01 00:00:00,31374.377264,2019-02-01 00:00:00,32021.03,2019-02-01 00:00:00,31872.179211,2019-02-01 00:00:00,32126.739766,2019-02-01 00:00:00,31948.68315,2019-02-01 00:00:00,31312.281942,2019-02-01 00:00:00,31368.284964,2019-02-01 00:00:00,31339.433448,2019-02-01 00:00:00,31421.093027,2019-02-01 00:00:00,31503.037,2019-02-01 00:00:00,31493.430,2019-02-01 00:00:00,31161.780355,2019-02-01 00:00:00,32245.453202
1,2019-02-01 01:00:00,30353.806407,2019-02-01 01:00:00,29554.05,2019-02-01 01:00:00,29685.964658,2019-02-01 01:00:00,30105.80,2019-02-01 01:00:00,30060.616031,2019-02-01 01:00:00,30503.032726,2019-02-01 01:00:00,29979.44233,2019-02-01 01:00:00,29321.171197,2019-02-01 01:00:00,29342.353761,2019-02-01 01:00:00,29885.517565,2019-02-01 01:00:00,29421.283832,2019-02-01 01:00:00,29815.424,2019-02-01 01:00:00,29617.531,2019-02-01 01:00:00,29447.620186,2019-02-01 01:00:00,30317.336597
2,2019-02-01 02:00:00,29429.213104,2019-02-01 02:00:00,28308.65,2019-02-01 02:00:00,28505.461085,2019-02-01 02:00:00,28730.71,2019-02-01 02:00:00,28791.507529,2019-02-01 02:00:00,29337.249341,2019-02-01 02:00:00,28720.68574,2019-02-01 02:00:00,28112.010835,2019-02-01 02:00:00,28313.998802,2019-02-01 02:00:00,28391.046167,2019-02-01 02:00:00,28539.136838,2019-02-01 02:00:00,28635.771,2019-02-01 02:00:00,28455.043,2019-02-01 02:00:00,28451.395711,2019-02-01 02:00:00,28718.710641
3,2019-02-01 03:00:00,28904.205359,2019-02-01 03:00:00,27601.48,2019-02-01 03:00:00,27855.221933,2019-02-01 03:00:00,27957.44,2019-02-01 03:00:00,28075.676511,2019-02-01 03:00:00,28681.681110,2019-02-01 03:00:00,28046.44589,2019-02-01 03:00:00,27611.726344,2019-02-01 03:00:00,27380.407098,2019-02-01 03:00:00,27894.855316,2019-02-01 03:00:00,27672.298537,2019-02-01 03:00:00,27855.768,2019-02-01 03:00:00,27712.234,2019-02-01 03:00:00,27704.391751,2019-02-01 03:00:00,27928.743341
4,2019-02-01 04:00:00,28714.921348,2019-02-01 04:00:00,27346.52,2019-02-01 04:00:00,27727.815649,2019-02-01 04:00:00,27802.84,2019-02-01 04:00:00,27896.662105,2019-02-01 04:00:00,28550.613245,2019-02-01 04:00:00,27738.71164,2019-02-01 04:00:00,27145.591865,2019-02-01 04:00:00,27480.197660,2019-02-01 04:00:00,27416.535458,2019-02-01 04:00:00,27332.009369,2019-02-01 04:00:00,27631.098,2019-02-01 04:00:00,27759.875,2019-02-01 04:00:00,27392.449683,2019-02-01 04:00:00,27679.824148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9523,2020-03-03 19:00:00,34433.450000,2020-03-03 19:00:00,35049.21,2020-03-03 19:00:00,38763.630766,2020-03-03 19:00:00,39283.70,2020-03-03 19:00:00,37640.730230,2020-03-03 19:00:00,38283.936762,2020-03-03 19:00:00,39055.31804,2020-03-03 19:00:00,36533.611538,2020-03-03 19:00:00,37024.468681,2020-03-03 19:00:00,36842.214179,2020-03-03 19:00:00,36960.207088,2020-03-03 19:00:00,37731.016,2020-03-03 19:00:00,38331.830,2020-03-03 19:00:00,35196.746675,2020-03-03 19:00:00,39194.639686
9524,2020-03-03 20:00:00,34324.041524,2020-03-03 20:00:00,34901.84,2020-03-03 20:00:00,38428.945844,2020-03-03 20:00:00,38791.40,2020-03-03 20:00:00,37280.720755,2020-03-03 20:00:00,37866.571236,2020-03-03 20:00:00,38426.28253,2020-03-03 20:00:00,36400.613389,2020-03-03 20:00:00,37347.225005,2020-03-03 20:00:00,37298.866743,2020-03-03 20:00:00,37127.922048,2020-03-03 20:00:00,38224.590,2020-03-03 20:00:00,37396.150,2020-03-03 20:00:00,34905.761724,2020-03-03 20:00:00,38307.274402
9525,2020-03-03 21:00:00,33821.654912,2020-03-03 21:00:00,34225.14,2020-03-03 21:00:00,37491.797795,2020-03

In [13]:
preds=preds[["Datetime_0"]+[i for i in preds.columns if not i.startswith("Datetime")]].rename(columns={"Datetime_0":"Datetime"})
preds

,Datetime,LR_2,naive_2,LR_2_7_w_date,naive_7,LR_2_7,LR_7,ygur_rima_preds_datetime,Cat1_48hour,Cat2_categorical_48hour,Cat1_24hour,Cat2_categorical_24hour,Keras_Basic_dense_24,Keras_Basic_dense_48,HW_preds_48hour,HW_preds_24hour
0,2019-02-01 00:00:00,31744.801869,31427.68,31374.377264,32021.03,31872.179211,32126.739766,31948.68315,31312.281942,31368.284964,31339.433448,31421.093027,31503.037,31493.430,31161.780355,32245.453202
1,2019-02-01 01:00:00,30353.806407,29554.05,29685.964658,30105.80,30060.616031,30503.032726,29979.44233,29321.171197,29342.353761,29885.517565,29421.283832,29815.424,29617.531,29447.620186,30317.336597
2,2019-02-01 02:00:00,29429.213104,28308.65,28505.461085,28730.71,28791.507529,29337.249341,28720.68574,28112.010835,28313.998802,28391.046167,28539.136838,28635.771,28455.043,28451.395711,28718.710641
3,2019-02-01 03:00:00,28904.205359,27601.48,27855.221933,27957.44,28075.676511,28681.681110,28046.44589,27611.726344,27380.407098,27894.855316,27672.298537,27855.768,27712.234,27704.391751,27928.743341
4,2019-02-01 04:00:00,28714.921348,27346.52,27727.815649,27802.84,27896.662105,28550.613245,27738.71164,27145.591865,27480.197660,27416.535458,27332.009369,27631.098,27759.875,27392.449683,27679.824148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9523,2020-03-03 19:00:00,34433.450000,35049.21,38763.630766,39283.70,37640.730230,38283.936762,39055.31804,36533.611538,37024.468681,36842.214179,36960.207088,37731.016,38331.830,35196.746675,39194.639686
9524,2020-03-03 20:00:00,34324.041524,34901.84,38428.945844,38791.40,37280.720755,37866.571236,38426.28253,36400.613389,37347.225005,37298.866743,37127.922048,38224.590,37396.150,34905.761724,38307.274402
9525,2020-03-03 21:00:00,33821.654912,34225.14,37491.797795,37588.86,36301.219739,36847.073493,37493.64405,35472.911196,36789.700181,36169.047983,36589.808965,37079.137,36533.484,34102.138172,37190.286189
9526,2020-03-03 22:00:00,33102.945835,33257.06,36424.989624,36290.64,35168.705158,35746.459492,36200.27294,34919.122612,35609.855302,35232.776345,35744.313864,35741.207,35341.290,33085.861894,35839.197807


In [14]:
preds.isna().sum()

Datetime                    0
LR_2                        0
naive_2                     0
LR_2_7_w_date               0
naive_7                     0
LR_2_7                      0
LR_7                        0
ygur_rima_preds_datetime    0
Cat1_48hour                 0
Cat2_categorical_48hour     0
Cat1_24hour                 0
Cat2_categorical_24hour     0
Keras_Basic_dense_24        0
Keras_Basic_dense_48        0
HW_preds_48hour             0
HW_preds_24hour             0
dtype: int64

In [15]:
# bruuuuuuuh
cons_holiday_binary=pd.read_csv("RealTimeConsumption_holiday.csv",encoding='latin-1')
cons_holiday_binary["Datetime"]=cons_holiday_binary["Date"]+" "+cons_holiday_binary["Hour"]
cons_holiday_binary["Datetime"]=pd.to_datetime(cons_holiday_binary["Datetime"],format="%d.%m.%Y %H:%M")
cons_holiday_binary.head()

,Date,Hour,Consumption,Datetime,is_annelergunu,is_arife,is_babalargunu,is_cocukbayrami,is_cumhuriyet,is_elections,is_examination,is_genclikbayrami,is_holiday,is_iscibayrami,is_kadinlargunu,is_kandil,is_kurbanbayrami,is_kurbanbayramilkgun,is_kurbanoncesigunler,is_onbestemmuz,is_outlierdays,is_ramazanbayrami,is_ramazanbayramilkgun,is_ramazanbayramisonrasi,is_ramazangunler,is_ramazanilkgun,is_ramazanoncesigunler,is_school_days,is_school_end,is_school_start,is_semester,is_semester_days,is_sevgililergunu,is_single_day_holidays,is_summerTimes,is_summer_break,is_uzunbayram,is_winterTimes,is_winter_break,is_winter_break_schools,is_yilbasi,is_yilbasioncesi,is_zaferbayrami
0,31.12.2015,00:00,29590.88,2015-12-31 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,31.12.2015,01:00,27785.43,2015-12-31 01:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,31.12.2015,02:00,26516.53,2015-12-31 02:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,31.12.2015,03:00,26091.62,2015-12-31 03:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,31.12.2015,04:00,25871.54,2015-12-31 04:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [16]:
preds["reel"]=cons_holiday_binary.loc[cons_holiday_binary["Datetime"]>=pd.to_datetime("2019-02-01 00:00:00")][["Consumption"]].values
preds.Datetime=pd.to_datetime(preds.Datetime)
preds

,Datetime,LR_2,naive_2,LR_2_7_w_date,naive_7,LR_2_7,LR_7,ygur_rima_preds_datetime,Cat1_48hour,Cat2_categorical_48hour,Cat1_24hour,Cat2_categorical_24hour,Keras_Basic_dense_24,Keras_Basic_dense_48,HW_preds_48hour,HW_preds_24hour,reel
0,2019-02-01 00:00:00,31744.801869,31427.68,31374.377264,32021.03,31872.179211,32126.739766,31948.68315,31312.281942,31368.284964,31339.433448,31421.093027,31503.037,31493.430,31161.780355,32245.453202,31878.29
1,2019-02-01 01:00:00,30353.806407,29554.05,29685.964658,30105.80,30060.616031,30503.032726,29979.44233,29321.171197,29342.353761,29885.517565,29421.283832,29815.424,29617.531,29447.620186,30317.336597,30064.08
2,2019-02-01 02:00:00,29429.213104,28308.65,28505.461085,28730.71,28791.507529,29337.249341,28720.68574,28112.010835,28313.998802,28391.046167,28539.136838,28635.771,28455.043,28451.395711,28718.710641,28813.77
3,2019-02-01 03:00:00,28904.205359,27601.48,27855.221933,27957.44,28075.676511,28681.681110,28046.44589,27611.726344,27380.407098,27894.855316,27672.298537,27855.768,27712.234,27704.391751,27928.743341,27865.76
4,2019-02-01 04:00:00,28714.921348,27346.52,27727.815649,27802.84,27896.662105,28550.613245,27738.71164,27145.591865,27480.197660,27416.535458,27332.009369,27631.098,27759.875,27392.449683,27679.824148,27554.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9523,2020-03-03 19:00:00,34433.450000,35049.21,38763.630766,39283.70,37640.730230,38283.936762,39055.31804,36533.611538,37024.468681,36842.214179,36960.207088,37731.016,38331.830,35196.746675,39194.639686,37936.79
9524,2020-03-03 20:00:00,34324.041524,34901.84,38428.945844,38791.40,37280.720755,37866.571236,38426.28253,36400.613389,37347.225005,37298.866743,37127.922048,38224.590,37396.150,34905.761724,38307.274402,37553.37
9525,2020-03-03 21:00:00,33821.654912,34225.14,37491.797795,37588.86,36301.219739,36847.073493,37493.64405,35472.911196,36789.700181,36169.047983,36589.808965,37079.137,36533.484,34102.138172,37190.286189,36581.73
9526,2020-03-03 22:00:00,33102.945835,33257.06,36424.989624,36290.64,35168.705158,35746.459492,36200.27294,34919.122612,35609.855302,35232.776345,35744.313864,35741.207,35341.290,33085.861894,35839.197807,35434.36


In [0]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [18]:
mapes={}
calc_mapes=preds.set_index("Datetime")
for model_name in calc_mapes.columns[:-1]:
    y_true,y_pred=calc_mapes[["reel",model_name]].values.T
    mape_=mean_absolute_percentage_error(y_true,y_pred)
    mapes[model_name]=mape_
pd.DataFrame([list(mapes.keys()),list(mapes.values())]).T.sort_values(1)

,0,1
9,Cat1_24hour,2.61997
11,Keras_Basic_dense_24,2.8726
10,Cat2_categorical_24hour,2.94813
7,Cat1_48hour,2.99118
8,Cat2_categorical_48hour,3.24907
6,ygur_rima_preds_datetime,3.46533
12,Keras_Basic_dense_48,3.68307
2,LR_2_7_w_date,4.05324
4,LR_2_7,4.98923
3,naive_7,5.20805


In [19]:
## 5 iq move düzelt bunu
best_models=[]
for i,row in preds.iterrows():
    best_mape=999999
    best_model=""
    for col in preds.columns.difference(["Datetime","reel"]):
        mape_=mean_absolute_percentage_error(row["reel"],row[col])
        if best_mape>mape_:
            best_mape=mape_
            best_model=col
    best_models.append(best_model)
preds["best_model"]=best_models
len(preds)+1

9529

In [0]:
mapes["best_case"]=preds.loc[preds["best_model"]!="unknown"].apply(lambda x: mean_absolute_percentage_error(x["reel"],x[x["best_model"]]),axis=1).mean()

In [21]:
mapes

{'Cat1_24hour': 2.6199679513147114,
 'Cat1_48hour': 2.991182739400095,
 'Cat2_categorical_24hour': 2.948131281186649,
 'Cat2_categorical_48hour': 3.2490686862665186,
 'HW_preds_24hour': 6.152474388551419,
 'HW_preds_48hour': 7.955785251367757,
 'Keras_Basic_dense_24': 2.8725964452793478,
 'Keras_Basic_dense_48': 3.683070359734644,
 'LR_2': 7.855694573000102,
 'LR_2_7': 4.989226864853979,
 'LR_2_7_w_date': 4.053236198257791,
 'LR_7': 5.22779264475642,
 'best_case': 0.6872404772042718,
 'naive_2': 8.15891679451568,
 'naive_7': 5.208054715113833,
 'ygur_rima_preds_datetime': 3.4653294353157422}

In [0]:
def brute_meta(n,buffer):
    shift_timespan=n*24
    chosen_models=["unknown"]*(shift_timespan+buffer)
    print("meta brute with n = {}, buffer = {}".format(n,buffer))
    for i in tqdm(range(shift_timespan,len(preds))):
        ## buraya prob koyacaggız
        chosen_models.append(preds["best_model"].iloc[i-shift_timespan:i].value_counts().index[0])
    preds["chosen"]=chosen_models[:len(chosen_models)-buffer]
    mapes_brute=preds.loc[preds["chosen"]!="unknown"].apply(lambda x: mean_absolute_percentage_error(x["reel"],x[x["chosen"]]),axis=1)
    mapes["window_{}_days_{}_buffer".format(n,buffer)]=mapes_brute.mean()
    print(mapes_brute.mean())

In [23]:
combins=np.concatenate([[(j,i) for i in [0,24,48]] for j in [5,7,14,28,30]])
for n,buffer in combins:
    brute_meta(n,buffer)

meta brute with n = 5, buffer = 0



3.408850416203426
meta brute with n = 5, buffer = 24



3.95668808178304
meta brute with n = 5, buffer = 48



3.9485640739674728
meta brute with n = 7, buffer = 0



3.033289980766917
meta brute with n = 7, buffer = 24


...


3.0795944880650645
meta brute with n = 28, buffer = 48



3.056409290568626
meta brute with n = 30, buffer = 0



3.026801674473698
meta brute with n = 30, buffer = 24



3.08247866415245
meta brute with n = 30, buffer = 48



3.0582706261034223


In [0]:
# shift_timespan=7*24
# chosen_models=["unknown"]*(shift_timespan)
# for i in tqdm(range(shift_timespan,len(preds))):
#     ## buraya prob koyacaggız
#     chosen_models.append(preds["best_model"].iloc[i-shift_timespan:i].value_counts().index[0])
# preds["chosen"]=chosen_models
# mapes_brute=preds.loc[preds["chosen"]!="unknown"].apply(lambda x: mean_absolute_percentage_error(x["reel"],x[x["chosen"]]),axis=1)
# mapes["window_7_days"]=mapes_brute.mean()
# mapes_brute.mean()
# 
# shift_timespan=5*24
# chosen_models=["unknown"]*(shift_timespan+48)
# for i in tqdm(range(shift_timespan,len(preds))):
#     ## buraya prob koyacaggız
#     chosen_models.append(preds["best_model"].iloc[i-shift_timespan:i].value_counts().index[0])
# preds["chosen_5"]=chosen_models[:-48]
# mapes_brute=preds.loc[preds["chosen_5"]!="unknown"].apply(lambda x: mean_absolute_percentage_error(x["reel"],x[x["chosen_5"]]),axis=1)
# mapes["window_5_days"]=mapes_brute.mean()
# 
# shift_timespan=14*24
# chosen_models=["unknown"]*(shift_timespan+48)
# for i in tqdm(range(shift_timespan,len(preds))):
#     chosen_models.append(preds["best_model"].iloc[i-shift_timespan:i].value_counts().index[0])
# preds["chosen_14"]=chosen_models[:-48]
# mapes_brute=preds.loc[preds["chosen_14"]!="unknown"].apply(lambda x: mean_absolute_percentage_error(x["reel"],x[x["chosen_14"]]),axis=1)
# mapes["window_14_days"]=mapes_brute.mean()
# 
# shift_timespan=30*24
# chosen_models=["unknown"]*(shift_timespan+48)
# for i in tqdm(range(shift_timespan,len(preds))):
#     chosen_models.append(preds["best_model"].iloc[i-shift_timespan:i].value_counts().index[0])
# preds["chosen_30"]=chosen_models[:-48]
# mapes_brute=preds.loc[preds["chosen_30"]!="unknown"].apply(lambda x: mean_absolute_percentage_error(x["reel"],x[x["chosen_30"]]),axis=1)
# mapes["window_30_days"]=mapes_brute.mean()

In [25]:
mapes

{'Cat1_24hour': 2.6199679513147114,
 'Cat1_48hour': 2.991182739400095,
 'Cat2_categorical_24hour': 2.948131281186649,
 'Cat2_categorical_48hour': 3.2490686862665186,
 'HW_preds_24hour': 6.152474388551419,
 'HW_preds_48hour': 7.955785251367757,
 'Keras_Basic_dense_24': 2.8725964452793478,
 'Keras_Basic_dense_48': 3.683070359734644,
 'LR_2': 7.855694573000102,
 'LR_2_7': 4.989226864853979,
 'LR_2_7_w_date': 4.053236198257791,
 'LR_7': 5.22779264475642,
 'best_case': 0.6872404772042718,
 'naive_2': 8.15891679451568,
 'naive_7': 5.208054715113833,
 'window_14_days_0_buffer': 2.939732944448012,
 'window_14_days_24_buffer': 2.997888910308739,
 'window_14_days_48_buffer': 2.9582858089665347,
 'window_28_days_0_buffer': 3.0181765251982773,
 'window_28_days_24_buffer': 3.0795944880650645,
 'window_28_days_48_buffer': 3.056409290568626,
 'window_30_days_0_buffer': 3.026801674473698,
 'window_30_days_24_buffer': 3.08247866415245,
 'window_30_days_48_buffer': 3.0582706261034223,
 'window_5_days_0_

In [26]:
nisan_24=preds.loc[preds["Datetime"].between(pd.to_datetime("2019-04-24 00:00:00"),pd.to_datetime("2019-04-25 00:00:00"))]
day_by_day_sort={}
for i,row in nisan_24.iterrows():
    mapes_a={}
    for col in nisan_24.columns[1:11]:
        mape_=mean_absolute_percentage_error(row["reel"],row[col])
        mapes_a[col]=mape_
    day_by_day_sort[row["Datetime"]]=mapes_a
day_by_day_sort

{Timestamp('2019-04-24 00:00:00'): {'Cat1_24hour': 3.9682948505904756,
  'Cat1_48hour': 3.2550492638705992,
  'Cat2_categorical_48hour': 4.0141117858170565,
  'LR_2': 1.8654968712021158,
  'LR_2_7': 4.078852493495687,
  'LR_2_7_w_date': 3.7699078135235085,
  'LR_7': 6.957455676398823,
  'naive_2': 1.3105580265177919,
  'naive_7': 6.193579003332738,
  'ygur_rima_preds_datetime': 6.636509101465669},
 Timestamp('2019-04-24 01:00:00'): {'Cat1_24hour': 4.498841079847919,
  'Cat1_48hour': 4.149418695961482,
  'Cat2_categorical_48hour': 3.9536312716998854,
  'LR_2': 2.948258726195324,
  'LR_2_7': 4.214070795705579,
  'LR_2_7_w_date': 4.332855355359548,
  'LR_7': 7.993531180302443,
  'naive_2': 2.02019839323456,
  'naive_7': 6.291836117680087,
  'ygur_rima_preds_datetime': 6.5047205108557495},
 Timestamp('2019-04-24 02:00:00'): {'Cat1_24hour': 3.358198148889057,
  'Cat1_48hour': 3.224480103978185,
  'Cat2_categorical_48hour': 3.527110735605522,
  'LR_2': 3.502619439506481,
  'LR_2_7': 3.677257

In [0]:
pd.DataFrame.from_dict(day_by_day_sort).to_csv("24_nisan_sorted.csv")

In [0]:
{key:pd.DataFrame.from_dict(day_by_day_sort[key],orient="index").sort_values(by=0).index.tolist() for key in day_by_day_sort.keys() }

{Timestamp('2019-04-24 00:00:00'): ['naive_2',
  'LR_2',
  'Cat1_48hour',
  'LR_2_7_w_date',
  'Cat1_24hour',
  'Cat2_categorical_48hour',
  'LR_2_7',
  'naive_7',
  'ygur_rima_preds_datetime',
  'LR_7'],
 Timestamp('2019-04-24 01:00:00'): ['naive_2',
  'LR_2',
  'Cat2_categorical_48hour',
  'Cat1_48hour',
  'LR_2_7',
  'LR_2_7_w_date',
  'Cat1_24hour',
  'naive_7',
  'ygur_rima_preds_datetime',
  'LR_7'],
 Timestamp('2019-04-24 02:00:00'): ['naive_2',
  'Cat1_48hour',
  'Cat1_24hour',
  'LR_2',
  'Cat2_categorical_48hour',
  'LR_2_7',
  'LR_2_7_w_date',
  'naive_7',
  'ygur_rima_preds_datetime',
  'LR_7'],
 Timestamp('2019-04-24 03:00:00'): ['Cat1_24hour',
  'Cat1_48hour',
  'naive_2',
  'Cat2_categorical_48hour',
  'LR_2_7',
  'LR_2',
  'LR_2_7_w_date',
  'ygur_rima_preds_datetime',
  'naive_7',
  'LR_7'],
 Timestamp('2019-04-24 04:00:00'): ['Cat1_24hour',
  'Cat1_48hour',
  'naive_2',
  'Cat2_categorical_48hour',
  'LR_2_7',
  'LR_2_7_w_date',
  'LR_2',
  'naive_7',
  'ygur_rima_pre

In [0]:
pd.DataFrame.from_dict({key:pd.DataFrame.from_dict(day_by_day_sort[key],orient="index").sort_values(by=0).index.tolist() for key in day_by_day_sort.keys() }).T

,0,1,2,3,4,5,6,7,8,9
2019-04-24 00:00:00,naive_2,LR_2,Cat1_48hour,LR_2_7_w_date,Cat1_24hour,Cat2_categorical_48hour,LR_2_7,naive_7,ygur_rima_preds_datetime,LR_7
2019-04-24 01:00:00,naive_2,LR_2,Cat2_categorical_48hour,Cat1_48hour,LR_2_7,LR_2_7_w_date,Cat1_24hour,naive_7,ygur_rima_preds_datetime,LR_7
2019-04-24 02:00:00,naive_2,Cat1_48hour,Cat1_24hour,LR_2,Cat2_categorical_48hour,LR_2_7,LR_2_7_w_date,naive_7,ygur_rima_preds_datetime,LR_7
2019-04-24 03:00:00,Cat1_24hour,Cat1_48hour,naive_2,Cat2_categorical_48hour,LR_2_7,LR_2,LR_2_7_w_date,ygur_rima_preds_datetime,naive_7,LR_7
2019-04-24 04:00:00,Cat1_24hour,Cat1_48hour,naive_2,Cat2_categorical_48hour,LR_2_7,LR_2_7_w_date,LR_2,naive_7,ygur_rima_preds_datetime,LR_7
2019-04-24 05:00:00,Cat1_48hour,Cat1_24hour,naive_2,LR_2_7,LR_2,Cat2_categorical_48hour,LR_2_7_w_date,naive_7,ygur_rima_preds_datetime,LR_7
2019-04-24 06:00:00,naive_2,Cat1_24hour,Cat1_48hour,LR_2_7,Cat2_categorical_48hour,LR_2,LR_2_7_w_date,naive_7,ygur_rima_preds_datetime,LR_7
2019-04-24 07:00:00,naive_2,Cat1_24hour,Cat1_48hour,Cat2_categorical_48hour,LR_2,LR_2_7,LR_2_7_w_date,ygur_rima_preds_datetime,naive_7,LR_7
2019-04-24 08:00:00,Cat2_categorical_48hour,Cat1_48hour,Cat1_24hour,LR_2,naive_2,LR_2_7,LR_2_7_w_date,LR_7,ygur_rima_preds_datetime,naive_7
2019-04-24 09:00:00,Cat2_categorical_48hour,Cat1_48hour,LR_2_7_w_date,LR_2_7,naive_2,LR_7,ygur_rima_preds_datetime,LR_2,Cat1_24hour,naive_7


In [0]:
mapes

{'Cat1_24hour': 2.6199679513147114,
 'Cat1_48hour': 2.991182739400095,
 'Cat2_categorical_24hour': 2.948131281186649,
 'Cat2_categorical_48hour': 3.2490686862665186,
 'HW_preds_24hour': 6.152474388551419,
 'HW_preds_48hour': 7.955785251367757,
 'Keras_Basic_dense_24': 2.8725964452793478,
 'Keras_Basic_dense_48': 3.683070359734644,
 'LR_2': 7.855694573000102,
 'LR_2_7': 4.989226864853979,
 'LR_2_7_w_date': 4.053236198257791,
 'LR_7': 5.22779264475642,
 'best_case': 0.6872404772042718,
 'naive_2': 8.15891679451568,
 'naive_7': 5.208054715113833,
 'window_14_days_0_buffer': 2.954443548681186,
 'window_14_days_24_buffer': 3.0125497747885506,
 'window_14_days_48_buffer': 2.973440981242279,
 'window_28_days_0_buffer': 3.0170898860443414,
 'window_28_days_24_buffer': 3.0811627135132427,
 'window_28_days_48_buffer': 3.0608280930514087,
 'window_30_days_0_buffer': 3.037566241685234,
 'window_30_days_24_buffer': 3.080684164685388,
 'window_30_days_48_buffer': 3.068768495302032,
 'window_5_days_0

In [0]:
pd.DataFrame.from_dict(mapes,orient="index").to_excel("Mapes.xlsx")

In [0]:
# preds.loc[1344,"Datetime"]=pd.to_datetime("2019-03-29 00:00:00")

In [0]:
np.unique([i.split("_")[1] for i in weather_data.columns if i!="Datetime"])

array(['Adana / Incirlik', 'Adana / Sakirpasa', 'Afyon', 'Akhisar',
       'Ankara / Esenboga', 'Ankara / Etimesgut', 'Antalya', 'Aydin',
       'Balikesir', 'Bandirma', 'Batman', 'Bodrum', 'Canakkale',
       'Dalaman', 'Diyarbakir', 'Elazig', 'Erzincan', 'Erzurum',
       'Eskisehir', 'Gaziantep', 'Iskenderun', 'Isparta',
       'Isparta / Gölba??', 'Istanbul / Ataturk',
       'Izmir / Adnan Menderes', 'Izmir / Cigli', 'Kars',
       'Kayseri / Erkilet', 'Konya', 'Malatya / Erhac', 'Merzifon',
       'Murted Tur-Afb', 'Sivas', 'Tokat', 'Trabzon', 'Usak', 'Van',
       'Zonguldak'], dtype='<U22')

## decision tree

In [0]:
weather_data=pd.read_csv('kubi_temp_humid_wind_data_20151231-2020-04-02.csv')
weather_data=weather_data.rename(columns={"rounded_date":"Datetime"})
weather_data["Datetime"]=pd.to_datetime(weather_data["Datetime"])
cons_holiday_binary=pd.merge(cons_holiday_binary,weather_data,on="Datetime",how="left")
cat_df_1=cons_holiday_binary[["Datetime"] + [i for i in cons_holiday_binary.columns if i.startswith("is_")]].copy()
cat_df_1["Hour"]=cat_df_1["Datetime"].apply(lambda x:x.hour).astype(int)
cat_df_1["Day"]=cat_df_1["Datetime"].apply(lambda x:x.day).astype(int)
cat_df_1["Month"]=cat_df_1["Datetime"].apply(lambda x:x.month).astype(int)
cat_df_1["weekday"]=cat_df_1["Datetime"].apply(lambda x: x.weekday()).astype(int)

In [0]:
train_df=pd.merge(preds.drop(columns=["chosen"]),cat_df_1,on="Datetime",how="left")

In [0]:
from sklearn.tree import DecisionTreeClassifier

def dt_window_approach(n,buffer):
    ## n'den baslayıp surkeli growing olarak gidiyor windows
    shift_timespan=n*24
    print("decision tree window approach with n = {}, buffer = {}".format(n,buffer))
    predicted_model=["unknown"]*(shift_timespan+buffer)
    ranking=["unknown"]*(shift_timespan+buffer)

    for i in tqdm(range(shift_timespan,len(preds)-buffer)):
        # buffer koy
        train_df_epoch=train_df.iloc[:i].reset_index()
        valid_df=train_df.iloc[i+buffer:i+buffer+1].reset_index()
        tree=DecisionTreeClassifier(max_depth=8,min_samples_leaf=10)
        tree.fit(train_df.drop(columns=["best_model","Datetime"]),train_df["best_model"])
        # random forestt rreggrressor

        pred=tree.predict(pd.DataFrame(valid_df).drop(columns=["best_model","Datetime"]))
        a=tree.classes_.tolist()
        b=tree.predict_proba(pd.DataFrame(valid_df).drop(columns=["best_model","Datetime"])).tolist()[0]
        ranking.append(dict(zip(a,b)))
        # predict proba
        predicted_model.append(pred[0])
    preds["tree_{}_n_{}_bufferpredicted".format(n,buffer)]=predicted_model
    return predicted_model,ranking

In [30]:
combins=np.concatenate([[(j,i) for i in [24,48]] for j in [7,14,28]])
rankings={}
predictions_={}
for n,buffer in combins:
    pred_model_list,ranking=dt_window_approach(n,buffer)
    rankings["n_{}_buffer_{}".format(n,buffer)]=ranking
    predictions_["n_{}_buffer_{}".format(n,buffer)]=pred_model_list

decision tree window approach with n = 7, buffer = 24


ValueError: ignored

In [0]:
preds

,Datetime,LR_2,naive_2,LR_2_7_w_date,naive_7,LR_2_7,LR_7,ygur_rima_preds_datetime,Cat1_48hour,Cat2_categorical_48hour,Cat1_24hour,Cat2_categorical_24hour,Keras_Basic_dense_24,Keras_Basic_dense_48,HW_preds_48hour,HW_preds_24hour,reel,best_model,chosen,tree_7_n_24_bufferpredicted,tree_7_n_48_bufferpredicted,tree_14_n_24_bufferpredicted,tree_14_n_48_bufferpredicted
0,2019-02-01 00:00:00,31744.801869,31427.68,31374.377264,32021.03,31872.179211,32126.739766,31948.68315,31312.281942,31368.284964,31339.433448,31421.093027,31503.037,31493.430,31161.780355,32245.453202,31878.29,LR_2_7,unknown,unknown,unknown,unknown,unknown
1,2019-02-01 01:00:00,30353.806407,29554.05,29685.964658,30105.80,30060.616031,30503.032726,29979.44233,29321.171197,29342.353761,29885.517565,29421.283832,29815.424,29617.531,29447.620186,30317.336597,30064.08,LR_2_7,unknown,unknown,unknown,unknown,unknown
2,2019-02-01 02:00:00,29429.213104,28308.65,28505.461085,28730.71,28791.507529,29337.249341,28720.68574,28112.010835,28313.998802,28391.046167,28539.136838,28635.771,28455.043,28451.395711,28718.710641,28813.77,LR_2_7,unknown,unknown,unknown,unknown,unknown
3,2019-02-01 03:00:00,28904.205359,27601.48,27855.221933,27957.44,28075.676511,28681.681110,28046.44589,27611.726344,27380.407098,27894.855316,27672.298537,27855.768,27712.234,27704.391751,27928.743341,27865.76,Keras_Basic_dense_24,unknown,unknown,unknown,unknown,unknown
4,2019-02-01 04:00:00,28714.921348,27346.52,27727.815649,27802.84,27896.662105,28550.613245,27738.71164,27145.591865,27480.197660,27416.535458,27332.009369,27631.098,27759.875,27392.449683,27679.824148,27554.48,Cat2_categorical_48hour,unknown,unknown,unknown,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9523,2020-03-03 19:00:00,34433.450000,35049.21,38763.630766,39283.70,37640.730230,38283.936762,39055.31804,36533.611538,37024.468681,36842.214179,36960.207088,37731.016,38331.830,35196.746675,39194.639686,37936.79,Keras_Basic_dense_24,ygur_rima_preds_datetime,Keras_Basic_dense_24,Keras_Basic_dense_48,Keras_Basic_dense_24,Keras_Basic_dense_48
9524,2020-03-03 20:00:00,34324.041524,34901.84,38428.945844,38791.40,37280.720755,37866.571236,38426.28253,36400.613389,37347.225005,37298.866743,37127.922048,38224.590,37396.150,34905.761724,38307.274402,37553.37,Keras_Basic_dense_48,ygur_rima_preds_datetime,Keras_Basic_dense_24,Keras_Basic_dense_48,Keras_Basic_dense_24,Keras_Basic_dense_48
9525,2020-03-03 21:00:00,33821.654912,34225.14,37491.797795,37588.86,36301.219739,36847.073493,37493.64405,35472.911196,36789.700181,36169.047983,36589.808965,37079.137,36533.484,34102.138172,37190.286189,36581.73,Cat2_categorical_24hour,ygur_rima_preds_datetime,Keras_Basic_dense_24,Keras_Basic_dense_48,Keras_Basic_dense_24,Keras_Basic_dense_48
9526,2020-03-03 22:00:00,33102.945835,33257.06,36424.989624,36290.64,35168.705158,35746.459492,36200.27294,34919.122612,35609.855302,35232.776345,35744.313864,35741.207,35341.290,33085.861894,35839.197807,35434.36,Keras_Basic_dense_48,ygur_rima_preds_datetime,LR_2_7_w_date,Keras_Basic_dense_48,LR_2_7_w_date,Keras_Basic_dense_48


In [0]:
fig=tree.fit(train_df.drop(columns=["best_model","Datetime"]),train_df["best_model"])
_, ax = plt.subplots(figsize=(15, 15))  # whatever size you want
plot_tree(fig,ax=ax,feature_names=train_df.drop(columns=["Datetime","best_model"]).columns,fontsize=16);

In [0]:
from sklearn.tree import DecisionTreeClassifier

shift_timespan=30*24
predicted_model=["unknown"]*shift_timespan
ranking=["unknown"]*shift_timespan
for i in tqdm(range(shift_timespan,len(preds),24)):
    # buffer koy
    train_df=preds[["Datetime","best_model"]].iloc[:i]
    train_df=pd.merge(train_df,cat_df_1,how="left",on="Datetime")
    valid_df=train_df.iloc[-24:]
    train_df=train_df.iloc[:i-24]
    tree=DecisionTreeClassifier(max_depth=16,min_samples_leaf=10)
    tree.fit(train_df.drop(columns=["best_model","Datetime"]),train_df["best_model"])
    # random forestt rreggrressor
    pred=tree.predict(pd.DataFrame(valid_df).drop(columns=["best_model","Datetime"]))
    # a=tree.classes_.tolist()
    # b=tree.predict_proba(pd.DataFrame(valid_df).T.drop(columns=["best_model","Datetime"])).tolist()[0]
    # ranking.append(dict(zip(a,b)))
    # predict proba
    predicted_model=np.concatenate((np.array(predicted_model),pred))
# preds["ranking"]=ranking
preds["tree_predicted"]=predicted_model
preds

In [0]:
# parameterler
# tahminleri de eklemek
# gün bazlı seyi kacırıyoruz saat saat baktıgımız için, günün bütününe dair

In [0]:
2.540852726450188

In [0]:
mapes_tree={}
for col in [i for i in preds.columns if "tree" in i ]:
    mapes_=preds.loc[preds[col]!="unknown"].apply(lambda x: mean_absolute_percentage_error(x["reel"],x[x[col]]),axis=1)
    mapes_tree[col]=mapes_.mean()

In [0]:
mapes_tree

{'tree_14_n_24_bufferpredicted': 3.3000047383774977,
 'tree_14_n_48_bufferpredicted': 4.33701975062609,
 'tree_7_n_24_bufferpredicted': 3.2858896479918704,
 'tree_7_n_48_bufferpredicted': 4.331831216842988}

In [0]:
preds.iloc[[8024]]

In [0]:
preds[preds["tree_predicted"]!="unknown"]["tree_predicted"].value_counts()

In [0]:
preds

In [0]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


In [0]:
from sklearn.metrics import confusion_matrix
for col in ["chosen","chosen_5","chosen_14","tree_predicted"]:
    pah_="drive/My Drive/colab_datas/IE 492/kubi_preds/eval_plots/conf_matrices/"+col
    if not os.path.exists(pah_):
        os.mkdir(pah_)
    unq_pred=preds.loc[preds[col]!="unknown"].reset_index(drop=True)
    clss=labels=unq_pred[col].unique().tolist()
    y_true=unq_pred["best_model"].values
    y_pred=unq_pred[col].values
    cnf_matrix = confusion_matrix(y_true, y_pred,labels=clss)
    np.set_printoptions(precision=2)
    plt.figure(figsize=(15,15))
    plot_confusion_matrix(cnf_matrix, classes=clss,
                        title='Confusion matrix of {}, without normalization'.format(col))
    plt.savefig(pah_+"/conf_matrix.png")


In [0]:
preds["best_case"]=preds.apply(lambda x:x[x["best_model"]],axis=1)
preds["window_case"]=preds.apply(lambda x:x[x["chosen"]] if x["chosen"]!="unknown" else 0,axis=1)
preds["tree_case"]=preds.apply(lambda x:x[x["tree_predicted"]] if x["tree_predicted"]!="unknown" else 0,axis=1)

In [0]:
for col in preds.columns[1:-5]:
    eval_function(col,preds["reel"],preds[col],preds)

In [0]:
eval_function("best_case",preds["reel"],preds["best_case"],preds)

In [0]:
pred_wo_unk=preds.loc[preds["window_case"]!=0].reset_index(drop=True)
eval_function("meta_window_case",pred_wo_unk["reel"],pred_wo_unk["window_case"],pred_wo_unk)

In [0]:
pred_wo_unk=preds.loc[preds["tree_case"]!=0].reset_index(drop=True)
eval_function("meta_tree_case",pred_wo_unk["reel"],pred_wo_unk["tree_case"],pred_wo_unk)

In [0]:
pd.DataFrame.from_dict(mapes,orient="index")